In [5]:
import imageio
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from starter.utils import get_device, get_mesh_renderer, load_cow_mesh, get_points_renderer, unproject_depth_image
from starter.render_generic import load_rgbd_data
import mcubes
import pytorch3d
import torch
from tqdm import tqdm
from PIL import Image, ImageDraw


## Helper functions

In [2]:
def create_gif(images_list: list[np.ndarray], gif_path: Path, FPS=15):
    # images_list is a list of (H,W,3) images
    assert images_list[0].shape[2] == 3
    
    frame_duration_ms = 1000 // FPS
    imageio.mimsave(gif_path, images_list, duration=frame_duration_ms, loop=0)

## Question 1: Practicing with Cameras

### 1.1: 360-deg renders

In [3]:

device = get_device()
cow_path = "data/cow.obj"
image_size=256

# Get the renderer.
renderer = get_mesh_renderer(image_size=image_size)

# Get the vertices, faces, and textures.
vertices, faces = load_cow_mesh(cow_path)
vertices = vertices.unsqueeze(0)  # (N_v, 3) -> (1, N_v, 3)
faces = faces.unsqueeze(0)  # (N_f, 3) -> (1, N_f, 3)
textures = torch.ones_like(vertices)  # (1, N_v, 3)
textures = textures * torch.tensor([0.7, 0.7, 1])  # (1, N_v, 3)

mesh = pytorch3d.structures.Meshes(
    verts=vertices,
    faces=faces,
    textures=pytorch3d.renderer.TexturesVertex(textures),
)
mesh = mesh.to(device)


# Place a point light in front of the cow.
lights = pytorch3d.renderer.PointLights(location=[[0, 0, -3]], device=device)

images_list = []

for i in tqdm(range(0,360,10), desc="Rendering cow..."):

    theta = np.radians(i)
    c, s = np.cos(theta), np.sin(theta)
    R = torch.tensor([[c, 0, s], [0, 1, 0], [-s, 0, c]]).unsqueeze(0)

    # Prepare the camera:
    cameras = pytorch3d.renderer.FoVPerspectiveCameras(
        R=R, T=torch.tensor([[0, 0, 3]]), fov=60, device=device
    )
    
    rend = renderer(mesh, cameras=cameras, lights=lights)
    img = rend.cpu().numpy()[0, ..., :3]
        
    img *= 255
    img = img.astype('uint8')
    images_list.append(img)

create_gif(images_list, Path('out.gif'))

Rendering cow...: 100%|██████████| 36/36 [00:00<00:00, 70.04it/s]


## HW1Q1 result
<img src="out.gif" width="256" align="center">

## 1.2: Dolly Zoom

In [7]:

image_size=256
num_frames=10
duration=3
output_file="output/dolly.gif"
device = get_device()

mesh = pytorch3d.io.load_objs_as_meshes(["data/cow_on_plane.obj"])
mesh = mesh.to(device)
renderer = get_mesh_renderer(image_size=image_size, device=device)
lights = pytorch3d.renderer.PointLights(location=[[0.0, 0.0, -3.0]], device=device)

fovs = torch.linspace(5, 120, num_frames)

renders = []

width = 5
for fov in tqdm(fovs):
    # distance = 50
    distance = width / (2 * np.tan(0.5 * np.radians(fov))) # TODO: change this.
    T = [[0, 0, distance]]  # TODO: Change this.
    cameras = pytorch3d.renderer.FoVPerspectiveCameras(fov=fov, T=T, device=device)
    rend = renderer(mesh, cameras=cameras, lights=lights)
    rend = rend[0, ..., :3].cpu().numpy()  # (N, H, W, 3)
    renders.append(rend)

images = []
for i, r in enumerate(renders):
    image = Image.fromarray((r * 255).astype(np.uint8))
    draw = ImageDraw.Draw(image)
    draw.text((20, 20), f"fov: {fovs[i]:.2f}", fill=(255, 0, 0))
    images.append(np.array(image))

create_gif(images, Path('hw1q1p2.gif'))


  0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipykernel_12044/910618457.py:19: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  distance = width / (2 * np.tan(0.5 * np.radians(fov))) # TODO: change this.
100%|██████████| 10/10 [00:00<00:00, 177.39it/s]


## q1.2 Dolly Zoom Result
<img src="hw1q1p2.gif" width="256">